In [1]:
!pip install konlpy
!pip install tqdm

     |████████████████████████████████| 19.4 MB 4.8 MB/s 
     |████████████████████████████████| 448 kB 15.1 MB/s 


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f64c706cf10>)

In [4]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data[:5] # 상위 5개 출력

id                                           document  label
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1   8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2   4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1
3   9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1
4  10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1

In [5]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [6]:
print(train_data.isnull().values.any())

True


In [7]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인


False


In [8]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [9]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [10]:
train_data[:5] # 상위 5개 출력

id                                           document  label
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1   8132799  디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...      1
2   4655635                   폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고      1
3   9251303   와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지      1
4  10067386                         안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화      1

In [11]:
train_data['document'][0]

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [12]:
# 불용어 정의
stopwords = ['을', 'ㅋㅋ', '부터', '까지' '적', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

tokenized_data = []
train_data = train_data[0:30000]  #20만개 중 1만개
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.nouns(sentence) # 토큰화 #okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 30000/30000 [02:00<00:00, 249.90it/s]


In [13]:
len(tokenized_data)
tokenized_data[10]
#train_data['document'][10]

['내', '나이', '영화', '지금', '나', '감동', '훗날', '다시', '사하나', '그', '감정', '이해']

1. word2vec 적용

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 50, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(3796, 50)

In [ ]:
model.wv.vocab

In [ ]:
import numpy as np
np.sqrt(np.mean(model.wv.vectors[10]-model.wv.vectors[11])**2)

0.08997585624456406

In [ ]:
print(model.wv.most_similar("감독"))

[('소재', 0.9989410042762756), ('조연', 0.9989128112792969), ('두', 0.9988909363746643), ('구성', 0.9988313317298889), ('전개', 0.9988279342651367), ('시나리오', 0.9987644553184509), ('주연', 0.9987197518348694), ('각본', 0.9986777305603027), ('분위기', 0.9985643625259399), ('편집', 0.9985222220420837)]


In [ ]:
print(model.wv.most_similar("겨울왕국"))

[('어제', 0.9993469715118408), ('나중', 0.9992597103118896), ('보', 0.9992209076881409), ('재', 0.9992122054100037), ('새벽', 0.9992115497589111), ('제목', 0.9991993308067322), ('성인', 0.9991881847381592), ('언제', 0.9991864562034607), ('고', 0.9991788268089294), ('정', 0.9991533756256104)]


2. lda 적용

In [ ]:
tokenized_data

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle = True, random_state = 1, remove=('headers', 'footers', 'quotes'))
document = dataset.data
news_df = pd.DataFrame({'document':document})
# 데이터 전처리 과정
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
# 불용어 처리 및 토큰화
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [17]:
tokenized_doc = tokenized_data

In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
corpus[1] # 수행된 결과에서 두번째 문서 출력, 첫번째 문서의 인덱스는 0


[(4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 2),
 (24, 1),
 (25, 1)]

In [19]:
dictionary[66]
len(dictionary)


17185

In [20]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.058*"드라마" + 0.024*"아이" + 0.023*"우리" + 0.015*"편"')
(1, '0.053*"영화" + 0.036*"수" + 0.035*"볼" + 0.027*"가슴"')
(2, '0.199*"영화" + 0.056*"정말" + 0.025*"생각" + 0.019*"감동"')
(3, '0.046*"사랑" + 0.025*"대박" + 0.025*"말" + 0.021*"명작"')
(4, '0.060*"감동" + 0.029*"굿" + 0.025*"완전" + 0.017*"아주"')
(5, '0.086*"최고" + 0.074*"영화" + 0.031*"역시" + 0.018*"나"')
(6, '0.058*"영화" + 0.039*"진짜" + 0.036*"평점" + 0.034*"때"')
(7, '0.031*"스토리" + 0.014*"이야기" + 0.013*"진짜" + 0.012*"영상"')
(8, '0.094*"점" + 0.023*"그냥" + 0.017*"평점" + 0.014*"인간"')
(9, '0.078*"연기" + 0.045*"배우" + 0.035*"재미" + 0.019*"내용"')


In [21]:
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

     |████████████████████████████████| 1.7 MB 24.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=d83ddf6a54ad069a5731aa06f2226f42fc01f58860cb0413cba69e84adcb12bb
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [22]:
import re
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric
import logging
import pickle 
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt 

In [ ]:
coherence_values = [] 
model_list = [] 
iters = [6,10, 14, 18, 22, 26]

for num_topics in iters: 
     model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics) 
     model_list.append(model) 
     coherencemodel = CoherenceModel(model=model, texts=tokenized_doc, dictionary=dictionary, coherence='c_v') 
     coherence_values.append(coherencemodel.get_coherence()) 


In [29]:
for i in range(0, len(iters)):
  print(model_list[i], coherence_values[i])

LdaModel(num_terms=17185, num_topics=6, decay=0.5, chunksize=2000) 0.2997835352592961
LdaModel(num_terms=17185, num_topics=10, decay=0.5, chunksize=2000) 0.3113471902087504
LdaModel(num_terms=17185, num_topics=14, decay=0.5, chunksize=2000) 0.33064356638905495
LdaModel(num_terms=17185, num_topics=18, decay=0.5, chunksize=2000) 0.3402737588792746
LdaModel(num_terms=17185, num_topics=22, decay=0.5, chunksize=2000) 0.33711592756752323
LdaModel(num_terms=17185, num_topics=26, decay=0.5, chunksize=2000) 0.3593969812012115
